# Victorian LGA COVID cases timeseries

In [ ]:
# import libraries
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import datetime
from datetime import datetime

In [ ]:
# base url to scarpe
base_url = 'https://covidlive.com.au/vic/'

# read in list of Victorian LGA names
LGAs = pd.read_csv('../data/vic_LGAs.csv')
LGAs = LGAs['LGA'].tolist()

## Scrape the daily COVID data from https://covidlive.com.au/vic/ 

In [ ]:
# clean LGA names - repalce space with hyphen to append to base url
LGA_url = []
for l in LGAs:
    a = l.replace(" ", "-").lower()
    LGA_url.append(a)

In [ ]:
# scrape data from https://covidlive.com.au/vic/ for all LGAs
# table structure appears to change frequently
# may need to tweak code to account for changes in table structure 
columns = ["Date", "Cumulative_cases", "Daily_cases", "LGA_name", "Active" "Active_cases_change"]
master_df = pd.DataFrame(columns=columns)

# iterate over each LGA
for i in LGA_url:
    response = requests.get(base_url + i)
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table', {'class': 'DAILY-CASES-BY-LGA'})
    
    try:
        table_rows = table.find_all('tr')
        print("Retrieved URL " + i)
    except:
        print("No data for URL " + i)
    
    l = []
    for tr in table_rows:
        td = tr.find_all('td')
        row = [tr.text for tr in td]
        l.append(row)
        
    df = pd.DataFrame(l, columns=["Date", "-1", "Cumulative_cases", "-2", "Daily_cases", "Active_cases", "-3", "Active_cases_change"])
    del df['-1']
    del df['-2']
    del df['-3']
    #del df['Active_cases']
    df = df.drop(df.index[0])
    df['LGA_name'] = i
    master_df = pd.concat([master_df, df],ignore_index=True) 

In [ ]:
# delete dirty columns
# check the master dataframe
del master_df['ActiveActive_cases_change']
del master_df['Active_cases_change']
master_df.head(10)

covidlive.com.au formatted digits using a thousands comma which is nice for presentation, but can be a pain when scraping data. When converting to a dataframe, pandas has inferred the data as a string, not numeric. This will need to be cleaned and converted to numeric.

In [ ]:
# replace all commas with nothing
master_df['Cumulative_cases'] = master_df['Cumulative_cases'].str.replace(",", "")
master_df['Active_cases'] = master_df['Active_cases'].str.replace(",", "")

# convert columns to numeric
master_df['Cumulative_cases'] = pd.to_numeric(master_df['Cumulative_cases'])
master_df['Active_cases'] = pd.to_numeric(master_df['Active_cases'])

In [ ]:
# check the data types
master_df.dtypes

Check a couple of different LGAs, one single word and one with a hyphen, to ensure the data has been scraped correctly.

In [ ]:
# inspect a couple of LGAs to ensure data has been scraped correctly
# check Hume
master_df[master_df['LGA_name']== "wyndham"].head(5)

In [ ]:
# check Mount-Alexander
master_df[master_df['LGA_name']== "mount-alexander"].head(5)

I intend to use the values in the LGA name column as labels. As such, these need to be cleaned (i.e. hyphen removed).

In [ ]:
# update LGA where space was replaced with hypen for visualisation, and covert to proper case
master_df['LGA_name'] = master_df['LGA_name'].str.replace("-", " ").str.title()

# check LGAs updated
master_df[master_df['LGA_name'] == "Mount Alexander"].head(5)

Next, a flag is created to indentify which LGAs are part of greater Melbourne. While this information could be scraped, it was easier to manually pull these LGAs from https://en.wikipedia.org/wiki/Local_government_areas_of_Victoria

Finally, we run a quick count of LGA by region to check we've classified all LGAs.

In [ ]:
# add flag to each LGA indicating if it is metro or reginal
greater_melb = ['Melbourne',
'Port Phillip',
'Stonnington',
'Yarra',
'Banyule',
'Bayside',
'Boroondara',
'Darebin',
'Glen Eira',
'Hobsons Bay',
'Kingston',
'Manningham',
'Maribyrnong',
'Monash',
'Moonee Valley',
'Moreland',
'Whitehorse',
'Brimbank',
'Cardinia',
'Casey',
'Frankston',
'Greater Dandenong',
'Hume',
'Knox',
'Maroondah',
'Melton',
'Mornington Peninsula',
'Nillumbik',
'Whittlesea',
'Wyndham',
'Yarra Ranges']

# check count of LGA by region
master_df["Region"] = np.where(master_df["LGA_name"].isin(greater_melb), "Greater Melbourne", "Regional")
print(master_df.groupby('Region')['LGA_name'].nunique())

In [ ]:
# check the flag has been applied to the dataframe
master_df[master_df['Region'] == "Greater Melbourne"]

Next issue is the date. Currently, the date is represented in dd-mmm format - ideally we need this in a longer format so will convert to yyyy-mm-dd.

In [ ]:
# convert date time
master_df['Date'] = pd.to_datetime(master_df['Date'], format='%d %b')
master_df['Date'] = master_df['Date'].apply(lambda dt: dt.replace(year=2020))
master_df.head()

# Importing Shapefile

The next section of the notebook brings in the shapefile used to create the base layer of the map. The shapefile was sourced from the ABS: https://www.abs.gov.au/AUSSTATS/abs@.nsf/DetailsPage/1270.0.55.003July%202016?OpenDocument

* Read in the Victorian LGA shapefile
* Remove administrative LGAs note used for mapping
* Clean LGA names for merge with the master_df

In [ ]:
#import libraries
import matplotlib.pyplot as plt 
import geopandas as gpd
import shapefile as shp
import re

In [ ]:
# read in the shapefle of all Australian LGAs
sf_aus = gpd.read_file('../data/AUS_LGA_SHP/LGA_2020_AUST.shp')

In [ ]:
sf_aus.head(5)

In [ ]:
# subset aus shapefile to vic LGAs only
vic_sf = sf_aus[sf_aus['STE_NAME16'] == 'Victoria']

# remove LGA without polygons
vic_sf = vic_sf[vic_sf['LGA_NAME20'] != 'Migratory - Offshore - Shipping (Vic.)']
vic_sf = vic_sf[vic_sf['LGA_NAME20'] != 'No usual address (Vic.)']
vic_sf = vic_sf[vic_sf['LGA_NAME20'] != 'Unincorporated Vic']

In [ ]:
# remove text within parentheses
vic_sf['LGA_NAME20'] = vic_sf['LGA_NAME20'].str.replace(r"(\(.+\))", "")

# strip remaining whitespace from LGA name 
vic_sf['LGA_NAME20'] = vic_sf['LGA_NAME20'].str.rstrip()

# check LGAs have been cleaned
vic_sf[vic_sf['LGA_NAME20'] == "Wyndham"].head()

In [ ]:
# check the head of the dataframe
master_df.head()

#####################################################################################################################

# Merged the COVID data with the shapefile

Finally, we merge the COVID data with the VIC shapefile. This methodology was sources from the following TDS blog: https://towardsdatascience.com/lets-make-a-map-using-geopandas-pandas-and-matplotlib-to-make-a-chloropleth-map-dddc31c1983d


In [ ]:
# merge the vic_sf and covid data
merged = vic_sf.set_index('LGA_NAME20').join(master_df.set_index('LGA_name'))

# check the (n) of rows and columns
merged.shape

In [ ]:
# insepct the first few rows
merged.head()

In [ ]:
# clean the data frame
# remove rows with missing dates
df_plot = merged[merged['Date'].notna()]

# subset to only Greater Melbourne LGAs
df_plot = df_plot[df_plot['Region'] == 'Greater Melbourne']

# subset to only included required columns
df_plot = df_plot[['Cumulative_cases', 'Active_cases','geometry', 'Date', 'Region']]

# Create a single plot

The next section creates a snapshot plot of a single date to ensure the code works to create the cumulative COID-19 case count for each LGA. The test plot below uses data from 10 August 2020 to generate a single plot.

The colour scale can easily be changed to one of many matplotlib default scales. for more info see: https://matplotlib.org/3.1.0/tutorials/colors/colormaps.html

<img src="../images/colour_scales.png" style="width:500px;height:400px;">

The name of the 'colour' variable can be edited to change to the desired colour scale.

In [ ]:
# subset the data to a specific day
dfa = df_plot[df_plot['Date'] == '2020-08-10']

# colour pallette
colour = 'YlOrRd'

# set a variable that will call whatever column we want to visualise on the map
variable = 'Cumulative_cases'
# set the range for the choropleth
vmin, vmax = 0, 1600

# create figure and axes for Matplotlib
fig, ax = plt.subplots(1, figsize=(14, 8))
dfa.plot(column=variable, cmap=colour, linewidth=0.8, ax=ax, edgecolor='0.8')

# remove the axis
ax.axis('off')

# add a title
ax.set_title('Cumulative COVID-19 cases for Greater Melbourne LGAs', fontdict={'fontsize': '18','fontweight' : '3'})

# create an annotation for the  data source
ax.annotate('Data source: https://covidlive.com.au',
           xy=(0.1, .08), xycoords='figure fraction',
           horizontalalignment='left', verticalalignment='top',
           fontsize=10, color='#555555')

# Create colorbar as a legend
sm = plt.cm.ScalarMappable(cmap=colour, norm=plt.Normalize(vmin=vmin, vmax=vmax))
sm._A = []
cbar = fig.colorbar(sm)
fig.savefig('../plots/single_plot/2020-08-10_cumulative_cases.png', dpi=300)

# Loop to create plots for each day

Finally we extract all dates from the master df dataframe. This will be used to iterate over each date and subset the dataframe to date [i]. We will generate one plot for each date and output the plot to the destinctation file folder. from there, the pots will be stitched together to create a GIF or video. 

Again, the bulk of the code to create plots for each day was adapted from: https://towardsdatascience.com/lets-make-a-map-using-geopandas-pandas-and-matplotlib-to-make-a-chloropleth-map-dddc31c1983d

In [ ]:
# reformat the dates in the covid data dataframe
#master_df['Date_only'] = [d.strftime("%Y-%m-%d") for d in master_df['Date']]

# extract a list of the unique dates - this will be used to iterate over in the for loop below
#dates = list(master_df['Date_only'].unique())

In [ ]:
# merge the covid data and the shapefile dataframes
#merged = vic_sf.set_index('LGA_NAME20').join(master_df.set_index('LGA_name'))
#merged = merged[merged['Region'] == 'Greater Melbourne']

In [ ]:
# remove NAs, subset to metro, remove columns
#df1 = merged[merged['Date'].notna()]
#df1 = df1[df1['Region'] == 'Greater Melbourne']
#df1 = df1[['Cumulative_cases', 'Active_cases','geometry', 'Date', 'Region']]

In [ ]:
# reformat dates 
#df1['Day'] = df1['Date'].dt.day
#df1['Month'] = df1['Date'].dt.month
#import calendar
#df1['Month'] = df1['Month'].apply(lambda x: calendar.month_abbr[x])

In [ ]:
# start the for loop to create one map per day
import os
import warnings
warnings.filterwarnings('ignore')

# set the range for the choropleth
vmin, vmax = 0, 1500
output_path = '/Users/nickkoleits/Documents/Projects/covid_vic_lga/plots/cumulative_cases'
variable = 'Cumulative_cases'
i = 1

colour = 'YlOrRd'

for date in dates:
    
    #subset data to each day
    data = df_plot[df_plot['Date'] == date]
    data['Cumulative_cases'] = pd.to_numeric(data['Cumulative_cases'])
    
    
    # create map, UDPATE: added plt.Normalize to keep the legend range the same for all maps
    fig = data.plot(column=variable, cmap=colour, figsize=(15,8), linewidth=0.8, edgecolor='0.8', vmin=vmin, vmax=vmax, legend=True, norm=plt.Normalize(vmin=vmin, vmax=vmax))
    
    # remove axis of chart
    fig.axis('off')
    
    # add a title
    fig.set_title('Cumulative COVID-19 cases\nGreater Melbourne - ' + str(date), fontdict={'fontsize': '20','fontweight' : '3'})

    # create an annotation for the  data source
    fig.annotate('Data source: https://covidlive.com.au',
        xy=(0.1, .08), xycoords='figure fraction',
        horizontalalignment='left', verticalalignment='top',
        fontsize=10, color='#555555')
    
    # this will save the figure as a high-res png in the output path. you can also save as svg if you prefer.
    filepath = os.path.join(output_path, str(date) +'_covid_cases.jpg')
    chart = fig.get_figure()
    chart.savefig(filepath, dpi=350)
    print("Saved image: " + output_path + str(date) +'_covid_cases.jpg')
    i = i + 1
    

# Create another set of plots using active cases

- We need to create a single plot to make sure we get all the plot features correct before running the loop. 
- Before that, we need to find the max a min values to ensure the plot colour scale is set to a sensible range - active cases and cumulative cases will have two very different ranges. 

In [ ]:
# get min and max values
active_max = df_plot['Active_cases'].max()
active_min = df_plot['Active_cases'].min()

In [ ]:
# subset the data to a specific day
dfa = df_plot[df_plot['Date'] == '2020-08-10']

# colour pallette
colour = 'YlOrRd'

# set a variable that will call whatever column we want to visualise on the map
variable = 'Active_cases'
# set the range for the choropleth
vmin, vmax = active_min, active_max

# create figure and axes for Matplotlib
fig, ax = plt.subplots(1, figsize=(14, 8))
dfa.plot(column=variable, cmap=colour, linewidth=0.8, ax=ax, edgecolor='0.8')

# remove the axis
ax.axis('off')

# add a title
ax.set_title('Cumulative COVID-19 cases for Greater Melbourne LGAs', fontdict={'fontsize': '18','fontweight' : '3'})

# create an annotation for the  data source
ax.annotate('Data source: https://covidlive.com.au',
           xy=(0.1, .08), xycoords='figure fraction',
           horizontalalignment='left', verticalalignment='top',
           fontsize=10, color='#555555')

# Create colorbar as a legend
sm = plt.cm.ScalarMappable(cmap=colour, norm=plt.Normalize(vmin=vmin, vmax=vmax))
sm._A = []
cbar = fig.colorbar(sm)
fig.savefig('../plots/single_plot/2020-08-10_active_cases.png', dpi=300)

In [ ]:
# start the for loop to create one map per day
import os
import warnings
warnings.filterwarnings('ignore')

# set the range for the choropleth
vmin, vmax = active_min, active_max
output_path = '../plots/active_cases'
variable = 'Active_cases'
i = 1

colour = 'YlOrRd'

for date in dates:
    
    #subset data to each day
    data = df_plot[df_plot['Date'] == date]
    data['Active_cases'] = pd.to_numeric(data['Active_cases'])
    
    
    # create map, UDPATE: added plt.Normalize to keep the legend range the same for all maps
    fig = data.plot(column=variable, cmap=colour, figsize=(15,8), linewidth=0.8, edgecolor='0.8', vmin=vmin, vmax=vmax, legend=True, norm=plt.Normalize(vmin=vmin, vmax=vmax))
    
    # remove axis of chart
    fig.axis('off')
    
    # add a title
    fig.set_title('Daily active COVID-19 cases\nGreater Melbourne - ' + str(date), fontdict={'fontsize': '20','fontweight' : '3'})

    # create an annotation for the  data source
    fig.annotate('Data source: https://covidlive.com.au',
        xy=(0.1, .08), xycoords='figure fraction',
        horizontalalignment='left', verticalalignment='top',
        fontsize=10, color='#555555')
    
    # this will save the figure as a high-res png in the output path. you can also save as svg if you prefer.
    filepath = os.path.join(output_path, str(date) +'_covid_cases.jpg')
    chart = fig.get_figure()
    chart.savefig(filepath, dpi=350)
    print("Saved image: " + output_path + str(date) +'_covid_cases.jpg')
    i = i + 1
    

# Producing the GIF

The GIF was created using https://gifmaker.me/
All source images were uploaded to the above url.